In [ ]:
import yaml
import pathlib
import anndata
import pandas as pd

In [ ]:
cluster_dir = '../L2/'

cluster_dir = pathlib.Path(cluster_dir).absolute()
cluster_dir.mkdir(exist_ok=True)

In [ ]:
parent_adata = anndata.read_h5ad('../L1/adata.with_coords.h5ad')

In [ ]:
def prepare_group(group, cells):
    group_dir = cluster_dir / group
    group_dir.mkdir(exist_ok=True)
    config_dir = group_dir / 'config'
    config_dir.mkdir(exist_ok=True)

    # subset cells
    with open(group_dir / 'select_cells.txt', 'w') as f:
        f.write('\n'.join(cells.index))

    for path in pathlib.Path('config/').glob('*.yaml'):
        with open(path) as f, open(config_dir / path.name, 'w') as outf:
            config = yaml.safe_load(f)

            # adjust the default parameters for all the groups
            if 'select_cells' in config:
                config['select_cells'] = 'select_cells.txt'
            if 'leiden_resolution' in config:
                config['leiden_resolution'] = 0.3

            yaml.safe_dump(config, outf)
    return

In [ ]:
for group, cells in parent_adata.obs.groupby('L1'):
    prepare_group(group, cells)

In [ ]:
# copy the snakefile
with open('../L2/Snakefile', 'w') as outf, open('Snakefile') as f:
    snakefile = f.read()
    snakefile = snakefile.replace('REPLACE_TEMPLATE_DIR',
                                  str(pathlib.Path().absolute()))
    outf.write(snakefile)